# Implementación de Modelos de ML con FastAPI

## Introducción

Este notebook educativo completo te guía paso a paso en la implementación de modelos de Machine Learning usando las herramientas más modernas de 2025: **FastAPI**, **uv**, **Docker**, y **Fly.io**. Aprenderás a crear un servicio web robusto para servir modelos de ML en producción.

### ¿Qué aprenderás?

- Configuración moderna de proyectos con **uv** (la alternativa rápida a pip/pipenv)
- Entrenamiento y guardado de modelos con **scikit-learn pipelines**
- Creación de APIs robustas con **FastAPI**
- Validación de datos con **Pydantic**
- Contenedorización con **Docker**
- Despliegue en la nube con **Fly.io**

### Caso de Uso: Predicción de Churn de Clientes

Implementaremos un modelo para predecir si un cliente cancelará su servicio (churn), un problema común en telecomunicaciones y servicios de suscripción.

## 1. Configuración del Entorno con UV

### ¿Qué es UV y por qué es tan rápido?

Imagina que estás construyendo algo con bloques de LEGO. `pip` es como un ayudante que va a la tienda a por cada pieza que necesitas, una por una. Si una pieza necesita otra más pequeña, tiene que volver a la tienda. Es fiable, pero puede llevar su tiempo.

**UV**, en cambio, es como un ayudante con un dron súper-rápido y una tablet. Antes de salir, mira tu lista, calcula al instante todas las piezas y sub-piezas que necesitarás, y las recoge todas de la tienda en un solo viaje a máxima velocidad.

En resumen, **UV es un instalador y gestor de entornos virtuales para Python, diseñado para ser extremadamente rápido**. Su objetivo es reemplazar a herramientas como `pip`, `pip-tools`, `venv` y `virtualenv` con una única interfaz de línea de comandos ultrarrápida.

### El Secreto de su Velocidad

La "magia" de UV no es una sola cosa, sino la combinación de tres factores clave:

1.  **Está escrito en Rust**: A diferencia de `pip` que está escrito en Python, UV está construido con Rust. Rust es un lenguaje de programación que compila a código máquina nativo, lo que le permite ejecutar tareas como la descarga e instalación de archivos a una velocidad mucho mayor que un lenguaje interpretado como Python. ¡Es como comparar un coche de Fórmula 1 (Rust) con un coche de calle (Python) para una carrera de velocidad!

2.  **Resolución de dependencias de última generación**: Cuando instalas un paquete (ej. `pandas`), este depende de otros (ej. `numpy`), que a su vez dependen de otros. Encontrar las versiones correctas que sean compatibles entre sí es un rompecabezas complejo. UV utiliza un algoritmo muy avanzado para resolver este "puzzle" de dependencias de forma increíblemente eficiente.

3.  **Un sistema de caché global e inteligente**: La primera vez que UV descarga un paquete, lo guarda en una caché global en tu sistema. La próxima vez que necesites ese mismo paquete en *otro proyecto*, UV no lo descarga de nuevo. Simplemente crea un enlace a la versión que ya tiene guardada. Esto hace que la creación de nuevos entornos sea casi instantánea.

> **Dato curioso**: El creador de UV, Charlie Marsh, es también el creador de **Ruff**, un *linter* de Python también escrito en Rust que es cientos de veces más rápido que sus predecesores.

### UV vs. Pip y otras herramientas

Pensar que UV es solo "un pip más rápido" es quedarse corto. La verdadera revolución es que **UV es una navaja suiza que reemplaza a un conjunto de herramientas**.

La forma tradicional de trabajar en Python requiere un equipo de varias herramientas:
* `venv` o `virtualenv`: Para crear y gestionar entornos virtuales aislados.
* `pip`: Para instalar los paquetes dentro de ese entorno.
* `pip-tools`: Una herramienta extra para compilar un `requirements.txt` a partir de un `pyproject.toml` y generar un archivo de bloqueo (`.txt`) que asegure la reproducibilidad.

UV integra todas estas funciones (y más) en un único ejecutable súper rápido.

Pensemos en una analogía: `pip` + `venv` es como tener una caja de herramientas con un martillo, un destornillador y una llave inglesa. Funcionan bien, pero tienes que ir cambiando de herramienta para cada tarea. **UV es como una multiherramienta Leatherman de última generación**: tienes todo lo que necesitas en un solo lugar, es más ligera y mucho más eficiente. 

### Tabla Comparativa Rápida

| Característica | `pip` + `venv` | `uv` |
| :--- | :--- | :--- |
| **Velocidad** | Moderada. La resolución de dependencias puede ser lenta. | **Extremadamente Rápida**. Gracias a Rust y su resolutor avanzado. |
| **Herramientas** | Múltiples (`python -m venv`, `pip`). | **Única y unificada** (un solo comando `uv`). |
| **Crear Entorno** | `python -m venv .venv` | `uv venv` (notablemente más rápido). |
| **Instalación** | `pip install pandas` | `uv pip install pandas` (sintaxis familiar). |
| **Caché de Paquetes**| El caché de pip es bueno, pero a veces inconsistente. | **Caché global e inteligente**. Acelera la creación de nuevos proyectos. |
| **Reproducibilidad**| Se necesita `pip-tools` para crear un archivo `.txt` de bloqueo. | **Soporte nativo**. Puede leer y generar archivos de bloqueo (`uv.lock`, `requirements.lock`). |

> La conclusión es simple: pasas de hacer malabares con 2 o 3 comandos a usar uno solo que, además, es entre **10 y 100 veces más rápido**. En entornos de Integración Continua (CI/CD), donde se crean y destruyen entornos constantemente, este ahorro de tiempo es gigantesco.


### Instalación y Configuración

```bash
# En tu terminal, instala uv (solo una vez)
curl -LsSf https://astral.sh/uv/install.sh | sh

# Crear nuevo proyecto
uv init mi-proyecto-ml
cd mi-proyecto-ml
```

> Otra alternativa: para usar `uv`, necesitas instalarlo en tu sistema. La forma más fácil es con `pip` normal: `pip install uv`).

### Primeros Pasos con UV

Aquí tienes el flujo de trabajo típico para un nuevo proyecto, paso a paso.

#### Paso 1: Crear el Entorno Virtual

Olvida el `python -m venv .venv`. Con UV, es más corto y mucho más rápido:

```bash
# Crear nuevo proyecto
uv init mi-proyecto-ml
cd mi-proyecto-ml

# Crea un entorno virtual en una carpeta llamada .venv
uv venv
```

¡Listo\! En una fracción de segundo, tendrás tu entorno creado. Si quisieras usar una versión específica de Python que tengas instalada, podrías hacer `uv venv -p 3.11`.

#### Paso 2: Activar el Entorno

Esta parte es **exactamente igual** a como siempre lo has hecho. UV crea una estructura de carpetas compatible.

```bash
# En Linux o macOS
source .venv/bin/activate

# En Windows (Command Prompt)
.venv\Scripts\activate
```

Una vez activado, tu terminal te mostrará `(.venv)` al principio de la línea.

#### Paso 3: Instalar Paquetes

La sintaxis es idéntica a la de `pip`, lo cual facilita enormemente la transición. Simplemente reemplazas `pip install` por `uv pip install`, otro comando valido es `uv add`.

```bash
# Instalar un solo paquete
uv pip install fastapi

# Instalar varios paquetes a la vez
uv pip install "pandas~=2.0" pydantic

# Instalar desde tu pyproject.toml
uv pip install -e .
```

Aquí notarás la diferencia más grande: la velocidad de descarga e instalación es asombrosa.

#### Paso 4: Generar un Archivo de Bloqueo

Este es el paso que garantiza que el entorno de todo tu equipo sea idéntico. `uv` lee tu `pyproject.toml` y genera un archivo `requirements.lock` con las versiones exactas de cada paquete.

```bash
# Lee pyproject.toml y crea un archivo de bloqueo
uv pip compile pyproject.toml -o requirements.lock
```

Este archivo `requirements.lock` es el que subirías a tu repositorio de Git.

#### Paso 5: Instalar desde el Archivo de Bloqueo

Ahora, imagina que eres un nuevo desarrollador que se une al proyecto. Tienes el `pyproject.toml` y el `requirements.lock`. Después de crear y activar tu entorno, solo necesitas un comando:

```bash
# Lee el archivo de bloqueo y sincroniza tu entorno.
# ¡Instala, elimina y actualiza paquetes para que coincida 100%!
uv sync requirements.lock
```

Este comando es increíblemente rápido y eficiente. Es el que usarías en tus flujos de CI/CD o para que un compañero se ponga al día.

### Estructura de Proyecto para ML

Esta es la estructura de directorios recomendada para el proyecto, siguiendo las mejores prácticas de desarrollo de software y MLOps.

```
mi-proyecto-ml/
├── .env                  # Variables de entorno y secretos
├── .gitignore
├── .python-version       # Versión de Python fijada para el proyecto
├── pyproject.toml        # Definición de dependencias y configuración
├── uv.lock               # Archivo de bloqueo para reproducibilidad
├── README.md
├── Dockerfile            # Instrucciones para la contenedorización
│
├── artifacts/            # Modelos entrenados, serializadores y otros artefactos
│   └── sentiment_model_v1.pkl
│
├── data/                 # Datasets del proyecto (ignorado por Git)
│   ├── raw/              # Datos originales, sin modificar
│   └── processed/        # Datos limpios y listos para el entrenamiento
│
├── notebooks/            # Jupyter Notebooks para exploración y análisis
│   └── 1.0-eda-initial-exploration.ipynb
│
├── src/                  # Código fuente de la aplicación
│   ├── __init__.py
│   ├── main.py           # Punto de entrada de la API (FastAPI)
│   ├── config.py         # Módulo de configuración
│   ├── api/              # Lógica de la API (endpoints)
│   ├── ml/               # Código de Machine Learning
│   └── schemas/          # Esquemas de datos (Pydantic)
│
├── tests/                # Pruebas automáticas
└── scripts/              # Scripts de utilidad (ej. para descargar datos)
```

### Explicación de la Estructura

La organización de este proyecto está diseñada para ser **clara, modular y escalable**. Cada directorio tiene una responsabilidad bien definida:

  * **Configuración (Raíz)**: Los archivos en la raíz del proyecto (`pyproject.toml`, `uv.lock`, `.python-version`, etc.) definen el entorno, las dependencias y las reglas del proyecto, asegurando que cualquier colaborador pueda replicar el entorno de desarrollo de forma idéntica.

  * **`src/` (Código Fuente)**: Es el corazón de la aplicación. Contiene todo el código Python que se ejecuta como parte del servicio final. La lógica está modularizada en subpaquetes como `api/`, `ml/` y `schemas/` para mantener el código organizado y fácil de mantener.

  * **`artifacts/` (Artefactos)**: Esta carpeta almacena los **productos generados por nuestro código**, no el código en sí. Su principal contenido son los modelos ya entrenados (ej. un archivo `.pkl` o `.h5`).

  * **`data/` (Datos)**: Un lugar centralizado para todos los datos necesarios. Se divide en `raw` para los datos originales e inmutables y `processed` para las versiones limpias y transformadas, listas para ser usadas en el entrenamiento. Esta carpeta se añade al `.gitignore` para evitar subir grandes volúmenes de datos al repositorio.

  * **`notebooks/` (Experimentación)**: Este es el "laboratorio". Contiene los Jupyter Notebooks usados para el Análisis Exploratorio de Datos (EDA), prototipado de modelos y visualizaciones. Separar los notebooks del código de producción en `src/` es crucial para mantener el proyecto limpio.

  * **`tests/` y `scripts/` (Soporte)**: `tests/` asegura la calidad y fiabilidad de nuestro código mediante pruebas automáticas, mientras que `scripts/` nos proporciona un lugar para herramientas de un solo uso que facilitan tareas de desarrollo.

Esta separación de responsabilidades hace que el proyecto sea más fácil de entender, probar, y finalmente, desplegar a producción.


### Configuración de Dependencias (pyproject.toml)

`pyproject.toml` es el cerebro detrás de la gestión de proyectos modernos en Python, y herramientas como **UV** están diseñadas para leerlo a la perfección.

Piensa en `pyproject.toml` como el **carné de identidad y el panel de control** de tu proyecto, todo en un único archivo.

Antes, la configuración de un proyecto de Python estaba repartida en varios archivos: `setup.py`, `requirements.txt`, `setup.cfg`, `MANIFEST.in`... ¡Era un poco caótico\! El archivo `pyproject.toml` fue introducido (en el [PEP 518](https://peps.python.org/pep-0518/)) para estandarizar y centralizar toda esa información en un solo lugar.

### ¿Qué hay dentro de un `pyproject.toml`?

Este archivo utiliza el formato [TOML](https://www.google.com/search?q=https://toml.io/es/) (Tom's Obvious, Minimal Language), que es muy fácil de leer para los humanos. Se organiza en secciones, pero nos centraremos en las más importantes para las dependencias.

Veamos un ejemplo práctico:

```toml
# Esta sección le dice a Python CÓMO construir tu proyecto.
# No necesitas preocuparte mucho por ella al principio.
[build-system]
requires = ["setuptools>=61.0"]
build-backend = "setuptools.build_meta"

# --- Aquí empieza lo interesante ---

# Esta es la "ficha de identidad" de tu proyecto.
[project]
name = "mi-proyecto-genial"
version = "0.1.0"
authors = [
  { name="Tu Nombre", email="tu@email.com" },
]
description = "Un pequeño proyecto de ejemplo."

# Aquí declaras las dependencias PRINCIPALES.
# Estas son las que se necesitan para que tu programa funcione.
dependencies = [
    "fastapi>=0.90.0", # Necesitamos fastapi, versión 0.90.0 o superior.
    "pandas",         # La última versión estable de pandas.
]

# Dependencias OPCIONALES. No son necesarias para todos los usuarios.
[project.optional-dependencies]
test = [
    "pytest",
    "pytest-cov",
]
docs = [
    "sphinx",
]

# En esta sección, otras herramientas pueden guardar su configuración.
# Por ejemplo, Ruff (el linter del que hablamos) se configura aquí.
[tool.ruff]
line-length = 88
```

Las dos secciones clave son:

1.  `[project.dependencies]`: Esta es tu lista principal de "ingredientes". Es el equivalente moderno al archivo `requirements.txt`. Aquí pones los paquetes que tu proyecto **necesita** para funcionar.
2.  `[project.optional-dependencies]`: Aquí defines grupos de dependencias para situaciones específicas. El caso más común es `test` (para instalar librerías de testing como `pytest`) o `dev` (para herramientas de desarrollo). Esto es genial porque alguien que solo quiere *usar* tu programa no necesita descargar todas las herramientas que tú usaste para *crearlo*.

### ¿Y cómo se relaciona esto con UV?

Aquí es donde todo encaja. **UV está diseñado para leer este archivo de forma nativa y ultrarrápida**.

  - Si ejecutas `uv pip install -e .` en la carpeta de tu proyecto, UV leerá la lista de `[project.dependencies]` y las instalará.
  - Si quieres instalar también las dependencias de testing, ejecutarías `uv pip install -e ".[test]"`. UV entenderá que debe instalar las dependencias principales **Y** las del grupo `test`.

Usar `pyproject.toml` centraliza toda la configuración, haciendo tu proyecto más limpio, reproducible y fácil de entender tanto para otros desarrolladores como para herramientas automáticas.

## 2. Generación de Datos Sintéticos para Churn

### Crear Dataset de Ejemplo

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

def create_churn_dataset(n_samples=10000, random_state=42):
    """
    Crear un dataset sintético realista para predicción de churn.
    
    Returns:
        DataFrame con características de clientes y etiquetas de churn
    """
    np.random.seed(random_state)
    
    # Crear datos base
    data = {
        'customer_id': range(1, n_samples + 1),
        'gender': np.random.choice(['Male', 'Female'], n_samples),
        'senior_citizen': np.random.choice([0, 1], n_samples, p=[0.84, 0.16]),
        'partner': np.random.choice(['Yes', 'No'], n_samples, p=[0.48, 0.52]),
        'dependents': np.random.choice(['Yes', 'No'], n_samples, p=[0.30, 0.70]),
        'tenure': np.random.randint(1, 73, n_samples),  # Meses
        'phone_service': np.random.choice(['Yes', 'No'], n_samples, p=[0.90, 0.10]),
        'internet_service': np.random.choice(['DSL', 'Fiber optic', 'No'], 
                                           n_samples, p=[0.34, 0.44, 0.22]),
        'online_security': np.random.choice(['Yes', 'No', 'No internet service'], 
                                          n_samples, p=[0.29, 0.50, 0.21]),
        'tech_support': np.random.choice(['Yes', 'No', 'No internet service'], 
                                       n_samples, p=[0.29, 0.50, 0.21]),
        'contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], 
                                   n_samples, p=[0.55, 0.21, 0.24]),
        'payment_method': np.random.choice([
            'Electronic check', 'Mailed check', 'Bank transfer (automatic)', 
            'Credit card (automatic)'
        ], n_samples, p=[0.34, 0.23, 0.22, 0.21]),
        'monthly_charges': np.round(np.random.normal(64.76, 30.0, n_samples), 2),
        'total_charges': np.round(np.random.normal(2283.30, 2266.77, n_samples), 2)
    }
    
    # Crear DataFrame
    df = pd.DataFrame(data)
    
    # Limpiar valores negativos en charges
    df['monthly_charges'] = df['monthly_charges'].clip(lower=18.25)
    df['total_charges'] = df['total_charges'].clip(lower=18.80)
    
    # Crear etiquetas de churn con lógica realista
    churn_probability = 0.2  # Baseline
    
    # Factores que aumentan churn
    tenure_factor = np.where(df['tenure'] < 12, 0.15, 0)  # Clientes nuevos
    contract_factor = np.where(df['contract'] == 'Month-to-month', 0.20, 0)  # Sin compromiso
    payment_factor = np.where(df['payment_method'] == 'Electronic check', 0.10, 0)  # Método de pago
    charges_factor = np.where(df['monthly_charges'] > 80, 0.08, 0)  # Cargos altos
    
    # Factores que reducen churn
    partner_factor = np.where(df['partner'] == 'Yes', -0.08, 0)  # Con pareja
    dependents_factor = np.where(df['dependents'] == 'Yes', -0.05, 0)  # Con dependientes
    long_tenure_factor = np.where(df['tenure'] > 48, -0.12, 0)  # Clientes antiguos
    
    # Calcular probabilidad final
    final_probability = (churn_probability + tenure_factor + contract_factor + 
                        payment_factor + charges_factor + partner_factor + 
                        dependents_factor + long_tenure_factor)
    
    # Generar etiquetas de churn
    df['churn'] = np.random.binomial(1, final_probability.clip(0.05, 0.85))
    
    return df

# Crear el dataset
print("Generando dataset sintético de churn...")
churn_data = create_churn_dataset(n_samples=10000)

print(f"Dataset creado:")
print(f"   • Muestras: {len(churn_data)}")
print(f"   • Características: {churn_data.shape[1]-2}")  # -2 para customer_id y churn
print(f"   • Tasa de churn: {churn_data['churn'].mean():.1%}")

# Mostrar primeras filas
churn_data.head()

## 3. Entrenamiento del Modelo con Pipelines de Scikit-learn

### ¿Por qué usar Pipelines?

Los **pipelines** de scikit-learn combinan múltiples pasos de preprocesamiento y modelado en un solo objeto, lo que:
- Simplifica el código
- Evita errores de data leakage
- Facilita la serialización
- Permite usar el modelo con datos en formato crudo

Pensemos en un **Pipeline** de Scikit-learn como una **receta de cocina** o una **línea de ensamblaje** para tu modelo de Machine Learning.

En lugar de realizar cada paso por separado (lavar los ingredientes, cortarlos, mezclarlos, hornearlos), un Pipeline te permite definir toda la secuencia de una vez. Le entregas los ingredientes crudos (tus datos) al principio, y al final obtienes el plato terminado (la predicción).

### Simplifica el Código

Imagina que necesitas rellenar valores faltantes y luego escalar tus datos antes de entrenar un modelo.

**Sin un Pipeline**, tu código se vería así, con pasos separados:

```python
# 1. Rellenar valores faltantes
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test) # ¡Ojo! Solo 'transform' en test

# 2. Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed) # De nuevo, solo 'transform'

# 3. Entrenar el modelo
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
```

Es fácil cometer errores, como aplicar `fit_transform` en el conjunto de prueba por accidente.

**Con un Pipeline**, todos esos pasos se encapsulan en uno solo:

```python
from sklearn.pipeline import Pipeline

# Definimos la "receta" completa
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')), # Paso 1: Rellenar
    ('scaler', StandardScaler()),              # Paso 2: Escalar
    ('model', LogisticRegression())            # Paso 3: Modelo
])

# Entrenamos todo el pipeline de una vez
pipeline.fit(X_train, y_train)
```

El código es más **limpio, corto y legible**.

### Evita Errores de "Data Leakage" (Fuga de Datos)

La **fuga de datos** es uno de los errores más peligrosos en Machine Learning. Ocurre cuando la información del conjunto de prueba (datos que el modelo "no debería haber visto") se "filtra" accidentalmente en el proceso de entrenamiento.

Piénsalo como si un estudiante **viera las respuestas del examen final mientras estudia**. Obviamente, sacará una nota perfecta en el examen, pero no habrá aprendido nada y no podrá resolver problemas nuevos.

Un Pipeline evita esto porque garantiza que cada paso (como el escalado de datos) se **ajuste (`fit`) únicamente con los datos de entrenamiento** y luego solo se **aplique (`transform`)** a los datos de prueba o a nuevos datos, imitando perfectamente las condiciones del mundo real.

### Facilita la Serialización (Guardar el Modelo)

Cuando quieres guardar tu trabajo, no solo necesitas el modelo, sino también todos los pasos de preprocesamiento que lo acompañan (el `imputer`, el `scaler`, etc.).

Sin un Pipeline, tendrías que guardar cada objeto por separado, lo cual es engorroso y propenso a errores. Con un Pipeline, **guardas un solo objeto** que contiene toda la secuencia de trabajo. Es como guardar el archivo de una receta completa en lugar de una lista desordenada de ingredientes y pasos.

```python
import joblib

# Guardas TODO el flujo de trabajo en un solo archivo
joblib.dump(pipeline, 'modelo_completo.pkl')

# Para cargarlo, es igual de simple
loaded_pipeline = joblib.load('modelo_completo.pkl')
```

### Permite Usar el Modelo con Datos Crudos

Esta es la consecuencia más práctica. Una vez que tu Pipeline está entrenado y guardado, puedes darle **datos nuevos y sin procesar** (datos "crudos"), y él se encargará de aplicar automáticamente toda la secuencia de preprocesamiento antes de hacer la predicción.

```python
# Datos nuevos, tal como llegan del mundo real
new_data = [[5.1, 3.5, None, 0.2]] # Tiene un valor faltante

# El pipeline se encarga de todo: imputa, escala y predice
prediction = loaded_pipeline.predict(new_data)
print(prediction)
```

Esto hace que poner tu modelo en producción sea **infinitamente más sencillo y seguro**.

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import joblib
import json
from datetime import datetime

def prepare_features(df):
    """
    Preparar características para el modelo.
    Convertir DataFrame a lista de diccionarios (formato requerido por DictVectorizer)
    """
    # Seleccionar características relevantes
    feature_columns = [
        'gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
        'phone_service', 'internet_service', 'online_security', 'tech_support',
        'contract', 'payment_method', 'monthly_charges', 'total_charges'
    ]
    
    # Convertir a lista de diccionarios
    X = df[feature_columns].to_dict('records')
    y = df['churn']
    
    return X, y, feature_columns

def train_churn_model(df, model_type='logistic_regression'):
    """
    Entrenar modelo de predicción de churn usando pipelines.
    
    Args:
        df: DataFrame con los datos
        model_type: Tipo de modelo ('logistic_regression' o 'random_forest')
    
    Returns:
        pipeline: Modelo entrenado
        metrics: Métricas de evaluación
    """
    print(f"🚀 Iniciando entrenamiento de modelo: {model_type}")
    
    # Preparar datos
    X, y, feature_columns = prepare_features(df)
    
    # Split train/validation/test (60/20/20)
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
    )
    
    print(f"📊 Split de datos:")
    print(f"   • Entrenamiento: {len(X_train)} muestras")
    print(f"   • Validación: {len(X_val)} muestras")
    print(f"   • Prueba: {len(X_test)} muestras")
    
    # Crear pipeline según el tipo de modelo
    if model_type == 'logistic_regression':
        pipeline = make_pipeline(
            DictVectorizer(sparse=False),
            LogisticRegression(
                random_state=42,
                max_iter=1000,
                class_weight='balanced'  # Manejar desbalance
            )
        )
    elif model_type == 'random_forest':
        pipeline = make_pipeline(
            DictVectorizer(sparse=False),
            RandomForestClassifier(
                n_estimators=100,
                random_state=42,
                class_weight='balanced',
                max_depth=10
            )
        )
    else:
        raise ValueError("model_type debe ser 'logistic_regression' o 'random_forest'")
    
    # Entrenar modelo
    print("🔄 Entrenando modelo...")
    pipeline.fit(X_train, y_train)
    
    # Evaluación en conjunto de validación
    y_val_pred = pipeline.predict(X_val)
    y_val_prob = pipeline.predict_proba(X_val)[:, 1]
    
    # Evaluación en conjunto de prueba
    y_test_pred = pipeline.predict(X_test)
    y_test_prob = pipeline.predict_proba(X_test)[:, 1]
    
    # Métricas
    val_auc = roc_auc_score(y_val, y_val_prob)
    test_auc = roc_auc_score(y_test, y_test_prob)
    
    metrics = {
        'model_type': model_type,
        'validation_auc': val_auc,
        'test_auc': test_auc,
        'train_samples': len(X_train),
        'val_samples': len(X_val),
        'test_samples': len(X_test),
        'feature_count': len(feature_columns),
        'training_date': datetime.now().isoformat()
    }
    
    print(f"\n✅ Entrenamiento completado!")
    print(f"📈 AUC Validación: {val_auc:.4f}")
    print(f"📈 AUC Prueba: {test_auc:.4f}")
    
    # Reporte detallado
    print(f"\n📋 Reporte de Clasificación (Conjunto de Prueba):")
    print(classification_report(y_test, y_test_pred, target_names=['No Churn', 'Churn']))
    
    return pipeline, metrics, (X_test, y_test)

# Entrenar modelos
print("=" * 60)
print("ENTRENAMIENTO DE MODELOS")
print("=" * 60)

# Modelo 1: Regresión Logística
lr_model, lr_metrics, (X_test, y_test) = train_churn_model(
    churn_data, 
    model_type='logistic_regression'
)

print("\n" + "=" * 60)

# Modelo 2: Random Forest
rf_model, rf_metrics, _ = train_churn_model(
    churn_data, 
    model_type='random_forest'
)

### Guardar Modelos Entrenados

In [ ]:
import os

def save_model_with_metadata(pipeline, metrics, model_name, models_dir="./models"):
    """
    Guardar modelo y sus metadatos de forma organizada.
    """
    # Crear directorio si no existe
    os.makedirs(models_dir, exist_ok=True)
    
    # Nombres de archivos
    model_filename = f"{model_name}_{datetime.now().strftime('%Y%m%d')}.joblib"
    metadata_filename = f"{model_name}_metadata.json"
    
    model_path = os.path.join(models_dir, model_filename)
    metadata_path = os.path.join(models_dir, metadata_filename)
    
    # Guardar modelo usando joblib (más eficiente que pickle para sklearn)
    joblib.dump(pipeline, model_path)
    
    # Guardar metadatos
    with open(metadata_path, 'w') as f:
        json.dump(metrics, f, indent=2, default=str)
    
    print(f"💾 Modelo guardado: {model_path}")
    print(f"📄 Metadatos guardados: {metadata_path}")
    
    return model_path, metadata_path

# Guardar ambos modelos
lr_model_path, lr_metadata_path = save_model_with_metadata(
    lr_model, lr_metrics, "churn_logistic_regression"
)

rf_model_path, rf_metadata_path = save_model_with_metadata(
    rf_model, rf_metrics, "churn_random_forest"
)

print("\n✅ Modelos guardados exitosamente!")

## 4. Creación de API con FastAPI

Piensa en una API (Interfaz de Programación de Aplicaciones) como un **camarero en un restaurante**. Tú (el cliente) no necesitas saber cómo funciona la cocina; solo le das tu pedido al camarero, él lo lleva a la cocina, y te trae el plato listo. La API hace exactamente eso, pero con datos.

FastAPI es un framework que te permite construir a ese "camarero" de una manera increíblemente eficiente, rápida y moderna.

### Velocidad

FastAPI está construido sobre dos pilares de alto rendimiento:

1.  **Starlette**: Es un microframework web ultrarrápido. FastAPI lo usa como su motor principal para manejar las peticiones web.
2.  **Pydantic**: Se encarga de la validación de datos y está escrito en parte en Rust, lo que lo hace extremadamente veloz.

Gracias a esto, FastAPI es uno de los frameworks de Python más rápidos que existen, comparable en rendimiento a aplicaciones escritas en lenguajes compilados como Go o Node.js. Esto significa que tu API puede atender a muchos más usuarios al mismo tiempo sin ralentizarse.

### Documentación Automática

Este es uno de los superpoderes de FastAPI. Imagina que cada vez que escribes el código de tu API, se **escribe solo un manual de instrucciones interactivo**.

FastAPI genera automáticamente una documentación en dos formatos:

  * **Swagger UI**
  * **ReDoc**

Solo tienes que ir a la URL `/docs` de tu API, y encontrarás una página donde puedes ver todos tus *endpoints* (las diferentes "órdenes" que tu camarero puede tomar), qué datos necesitan, y qué datos devuelven. ¡Incluso puedes probar la API directamente desde esa página\! Esto ahorra una cantidad enorme de tiempo en documentación y facilita el trabajo en equipo.

### Validación con Pydantic

Piensa en Pydantic como el **guardia de seguridad de tu API**. Antes de que cualquier dato entre a tu lógica, Pydantic lo revisa para asegurarse de que tiene el formato correcto.

Tú defines la "forma" de los datos que esperas usando clases de Python, y Pydantic se encarga del resto.

```python
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    price: float
    is_offer: bool | None = None
```

Si alguien intenta enviar un `price` que no es un número, FastAPI automáticamente le devolverá un error claro y descriptivo. Esto hace tu código mucho más seguro y robusto, evitando errores inesperados.

### Soporte Asíncrono (Async)

Imagina un chef que solo puede hacer una cosa a la vez (síncrono). Si está esperando que el agua hierva, no puede hacer nada más.

Un chef asíncrono, en cambio, pone el agua a hervir y, **mientras espera**, se pone a cortar las verduras. Es mucho más eficiente.

FastAPI te permite usar `async` y `await` para manejar operaciones que toman tiempo (como llamar a otra API o consultar una base de datos) sin bloquear todo el programa. Esto es fundamental para construir aplicaciones que necesitan manejar muchas conexiones simultáneas de manera eficiente.

### Tipado Nativo de Python (Type Hints)

FastAPI utiliza las **pistas de tipos** de Python (`str`, `int`, `bool`, etc.) para todo. No tienes que aprender una sintaxis nueva.

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/items/{item_id}")
def read_item(item_id: int, q: str | None = None):
    return {"item_id": item_id, "q": q}
```

Al declarar que `item_id` debe ser un `int`, FastAPI automáticamente:

1.  **Valida** que el dato recibido es un entero.
2.  **Documenta** que este endpoint espera un entero.
3.  Le da a tu editor de código (como VS Code) información para **autocompletar** y detectar errores mientras escribes.

En resumen, FastAPI usa características modernas de Python para darte una experiencia de desarrollo rápida, segura y muy agradable.

### Decoradores

Un **decorador** en Python es como ponerle un "sombrero" especial a una función para darle superpoderes o un nuevo comportamiento. Usas el símbolo `@` para aplicarlo.

En FastAPI, los decoradores le dicen a tu "camarero" (la API) qué hacer cuando alguien llega a una URL específica con un método de petición concreto (GET, POST, etc.).

### Los "Sombreros" de Operación: GET, POST, PUT, DELETE

Piensa en estos decoradores como las diferentes tareas que un camarero puede realizar: tomar una orden, entregar un plato, actualizar una orden o cancelarla.

  * **`@app.get("/ruta")`**: **Leer datos.** Se usa cuando un cliente quiere *obtener* información. Es como preguntar el menú del día. Es la operación más común.

    ```python
    @app.get("/items/{item_id}")
    def leer_item(item_id: int):
        # Aquí iría el código para buscar el item en una base de datos
        return {"item_id": item_id, "nombre": "Ejemplo de item"}
    ```

  * **`@app.post("/ruta")`**: **Crear datos.** Se usa cuando un cliente quiere *añadir* nueva información al sistema. Es como hacer un pedido nuevo en la cocina.

    ```python
    from pydantic import BaseModel

    class Item(BaseModel):
        name: str
        price: float

    @app.post("/items/")
    def crear_item(item: Item):
        # Aquí guardarías el nuevo 'item' en la base de datos
        return {"mensaje": f"Item '{item.name}' creado exitosamente."}
    ```

  * **`@app.put("/ruta")`**: **Actualizar datos.** Se usa para reemplazar o actualizar por completo un recurso existente. Es como cambiar tu pedido por completo.

    ```python
    @app.put("/items/{item_id}")
    def actualizar_item(item_id: int, item: Item):
        # Lógica para actualizar el item con el id correspondiente
        return {"item_id": item_id, **item.dict()}
    ```

  * **`@app.delete("/ruta")`**: **Borrar datos.** Se usa para eliminar un recurso. Es como cancelar un plato de tu orden.

    ```python
    @app.delete("/items/{item_id}")
    def borrar_item(item_id: int):
        # Lógica para borrar el item de la base de datos
        return {"mensaje": f"Item con id {item_id} ha sido eliminado."}
    ```


### Parámetros de Ruta y Consultas

FastAPI es inteligente y usa los argumentos de tu función para entender los datos que llegan:

1.  **Parámetros de Ruta (Path Parameters)**: Son valores que forman parte de la propia URL y se definen con llaves `{}`. En `@app.get("/items/{item_id}")`, `item_id` es un parámetro de ruta. FastAPI entiende que debe extraer ese valor de la URL y pasarlo a tu función.

2.  **Parámetros de Consulta (Query Parameters)**: Son parámetros opcionales que van al final de la URL después de un `?`. Por ejemplo, en `/users?role=admin`, `role` es un parámetro de consulta. Si un argumento de tu función **no** está en la ruta, FastAPI asume que es un parámetro de consulta.

    ```python
    @app.get("/users/")
    # 'limit' es un parámetro de consulta con un valor por defecto.
    # Se usaría así: /users/ o /users/?limit=50
    def leer_usuarios(limit: int = 100):
        # ...lógica para devolver usuarios...
        return {"limite": limit, "usuarios": []}
    ```

### El Cuerpo de la Petición (Request Body)

Para operaciones como `POST` y `PUT`, los datos suelen ser demasiado complejos para ir en la URL. En su lugar, se envían en el "cuerpo" de la petición, normalmente como un objeto JSON.

Aquí es donde usas un **modelo de Pydantic**. Al declarar un argumento de tu función con el tipo de un modelo Pydantic (como `item: Item`), le dices a FastAPI:

1.  Espera recibir un JSON en el cuerpo de la petición.
2.  Verifica que el JSON tenga la misma estructura que la clase `Item`.
3.  Si es válido, convierte el JSON en un objeto de Python y pásalo a mi función.
4.  Si no es válido, responde automáticamente con un error claro.

Esto hace que manejar datos de entrada complejos sea increíblemente simple y seguro.




### Aplicación FastAPI Principal

In [ ]:
# src/main.py
from fastapi import FastAPI, HTTPException, BackgroundTasks, Depends
from fastapi.middleware.cors import CORSMiddleware
from contextlib import asynccontextmanager
import joblib
import os
import logging
from typing import Dict, Any
from datetime import datetime

from schemas.predictions import CustomerInput, PredictionResponse, BatchPredictionRequest

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Almacenamiento global para modelos
ml_models: Dict[str, Any] = {}

@asynccontextmanager
async def lifespan(app: FastAPI):
    """Gestión del ciclo de vida de la aplicación"""
    logger.info("🚀 Iniciando carga de modelos...")
    
    try:
        # Cargar modelos disponibles
        models_dir = "models"
        if os.path.exists(models_dir):
            for filename in os.listdir(models_dir):
                if filename.endswith('.joblib'):
                    model_name = filename.replace('.joblib', '')
                    model_path = os.path.join(models_dir, filename)
                    
                    model = joblib.load(model_path)
                    ml_models[model_name] = model
                    logger.info(f"✅ Modelo cargado: {model_name}")
        
        if not ml_models:
            logger.warning("⚠️ No se encontraron modelos en el directorio")
        
        logger.info(f"📊 Total modelos cargados: {len(ml_models)}")
        
    except Exception as e:
        logger.error(f"❌ Error cargando modelos: {e}")
    
    yield
    
    # Limpieza al cerrar
    ml_models.clear()
    logger.info("🔄 Recursos liberados")

# Crear aplicación FastAPI
app = FastAPI(
    title="Churn Prediction API",
    description="""
    🎯 **API para Predicción de Churn de Clientes**
    
    Esta API utiliza modelos de Machine Learning para predecir la probabilidad
    de que un cliente cancele su servicio (churn).
    
    ## Características
    
    * **Predicciones individuales**: Predice churn para un cliente
    * **Predicciones por lotes**: Procesa múltiples clientes
    * **Múltiples modelos**: Soporte para diferentes algoritmos
    * **Validación automática**: Verificación de datos de entrada
    * **Documentación interactiva**: Swagger UI integrado
    
    ## Modelos Disponibles
    
    * **Regresión Logística**: Modelo interpretable y rápido
    * **Random Forest**: Modelo ensemble con alta precisión
    """,
    version="1.0.0",
    contact={
        "name": "Equipo ML",
        "email": "ml@tuempresa.com",
    },
    license_info={
        "name": "Apache 2.0",
        "url": "https://www.apache.org/licenses/LICENSE-2.0.html",
    },
    lifespan=lifespan
)

# Configurar CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # En producción, especifica dominios específicos
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Dependencia para obtener modelo
async def get_model(model_name: str = "churn_logistic_regression_20241201"):
    if model_name not in ml_models:
        available_models = list(ml_models.keys())
        raise HTTPException(
            status_code=404,
            detail=f"Modelo '{model_name}' no encontrado. Modelos disponibles: {available_models}"
        )
    return ml_models[model_name]

# === ENDPOINTS ===

@app.get("/", tags=["info"])
async def root():
    """Información básica de la API"""
    return {
        "message": "🎯 Churn Prediction API",
        "version": "1.0.0",
        "status": "active",
        "models_loaded": len(ml_models),
        "timestamp": datetime.now().isoformat()
    }

@app.get("/health", tags=["monitoring"])
async def health_check():
    """Health check para monitoreo"""
    return {
        "status": "healthy",
        "models_count": len(ml_models),
        "models_available": list(ml_models.keys()),
        "timestamp": datetime.now().isoformat()
    }

@app.get("/models", tags=["models"])
async def list_models():
    """Listar modelos disponibles"""
    model_info = {}
    
    for name, model in ml_models.items():
        model_info[name] = {
            "type": type(model).__name__,
            "steps": [step[0] for step in model.steps] if hasattr(model, 'steps') else "Pipeline",
        }
    
    return {
        "available_models": model_info,
        "total_count": len(ml_models)
    }

@app.post("/predict", response_model=PredictionResponse, tags=["predictions"])
async def predict_churn(
    customer: CustomerInput,
    background_tasks: BackgroundTasks,
    model_name: str = "churn_logistic_regression_20241201",
    model = Depends(get_model)
):
    """
    🎯 Predecir probabilidad de churn para un cliente
    
    Utiliza el modelo especificado para calcular la probabilidad de que
    el cliente cancele su servicio.
    
    - **customer**: Datos del cliente (ver esquema completo abajo)
    - **model_name**: Nombre del modelo a utilizar
    
    Retorna predicción, probabilidades y metadatos del modelo.
    """
    try:
        start_time = datetime.now()
        
        # Convertir datos de entrada a formato de diccionario
        customer_dict = customer.model_dump()
        
        # Hacer predicción
        prediction = model.predict([customer_dict])[0]
        probabilities = model.predict_proba([customer_dict])[0]
        
        # Calcular métricas
        churn_probability = float(probabilities[1])
        retention_probability = float(probabilities[0])
        confidence = max(probabilities)
        
        # Determinar categoría de riesgo
        if churn_probability >= 0.7:
            risk_category = "High"
        elif churn_probability >= 0.4:
            risk_category = "Medium"  
        else:
            risk_category = "Low"
        
        processing_time = (datetime.now() - start_time).total_seconds() * 1000
        
        response = PredictionResponse(
            prediction=int(prediction),
            churn_probability=churn_probability,
            retention_probability=retention_probability,
            risk_category=risk_category,
            confidence=float(confidence),
            model_info={
                "name": model_name,
                "type": type(model).__name__,
                "version": "1.0.0"
            },
            processing_time_ms=int(processing_time),
            timestamp=datetime.now()
        )
        
        # Log en background (no bloquea la respuesta)
        background_tasks.add_task(
            log_prediction,
            customer_dict,
            response.model_dump(),
            model_name
        )
        
        return response
        
    except Exception as e:
        logger.error(f"Error en predicción: {str(e)}")
        raise HTTPException(
            status_code=500,
            detail=f"Error procesando predicción: {str(e)}"
        )

# Función de logging asíncrono
async def log_prediction(customer_data: dict, prediction_result: dict, model_name: str):
    """Registrar predicción para monitoreo y análisis"""
    logger.info(
        f"PREDICTION - Model: {model_name}, "
        f"Churn_Prob: {prediction_result['churn_probability']:.3f}, "
        f"Risk: {prediction_result['risk_category']}, "
        f"Tenure: {customer_data.get('tenure', 'N/A')}"
    )

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=8000,
        reload=True,
        log_level="info"
    )

## 5. Modelos de Validación con Pydantic (...)

### Esquemas de Entrada y Salida

```python
# src/schemas/predictions.py
from pydantic import BaseModel, Field, validator
from typing import List, Optional, Dict, Any
from datetime import datetime
from enum import Enum

class RiskCategory(str, Enum):
    """Categorías de riesgo de churn"""
    LOW = "Low"
    MEDIUM = "Medium" 
    HIGH = "High"

class ContractType(str, Enum):
    """Tipos de contrato disponibles"""
    MONTH_TO_MONTH = "Month-to-month"
    ONE_YEAR = "One year"
    TWO_YEAR = "Two year"

class PaymentMethod(str, Enum):
    """Métodos de pago disponibles"""
    ELECTRONIC_CHECK = "Electronic check"
    MAILED_CHECK = "Mailed check"
    BANK_TRANSFER = "Bank transfer (automatic)"
    CREDIT_CARD = "Credit card (automatic)"

class InternetService(str, Enum):
    """Tipos de servicio de internet"""
    DSL = "DSL"
    FIBER_OPTIC = "Fiber optic"
    NO = "No"

class CustomerInput(BaseModel):
    """
    Modelo de entrada para datos del cliente.
    
    Todos los campos son validados automáticamente por Pydantic.
    """
    
    # Información demográfica
    gender: str = Field(
        ..., 
        description="Género del cliente",
        example="Male"
    )
    
    senior_citizen: int = Field(
        ..., 
        ge=0, 
        le=1,
        description="Es ciudadano senior (0=No, 1=Sí)",
        example=0
    )
    
    partner: str = Field(
        ...,
        description="Tiene pareja",
        example="Yes"
    )
    
    dependents: str = Field(
        ...,
        description="Tiene dependientes",
        example="No"
    )
    
    # Información del servicio
    tenure: int = Field(
        ...,
        ge=0,
        le=100,
        description="Meses como cliente",
        example=24
    )
    
    phone_service: str = Field(
        ...,
        description="Tiene servicio telefónico",
        example="Yes"
    )
    
    internet_service: InternetService = Field(
        ...,
        description="Tipo de servicio de internet",
        example="Fiber optic"
    )
    
    online_security: str = Field(
        ...,
        description="Tiene seguridad online",
        example="No"
    )
    
    tech_support: str = Field(
        ...,
        description="Tiene soporte técnico",
        example="Yes"
    )
    
    # Información contractual y financiera
    contract: ContractType = Field(
        ...,
        description="Tipo de contrato",
        example="Month-to-month"
    )
    
    payment_method: PaymentMethod = Field(
        ...,
        description="Método de pago",
        example="Electronic check"
    )
    
    monthly_charges: float = Field(
        ...,
        gt=0,
        lt=200,
        description="Cargos mensuales en USD",
        example=85.50
    )
    
    total_charges: float = Field(
        ...,
        ge=0,
        description="Total de cargos acumulados en USD",
        example=2052.00
    )
    
    # Campo opcional para ID del cliente
    customer_id: Optional[str] = Field(
        None,
        description="ID opcional del cliente",
        example="CUST001"
    )
    
    @validator('gender')
    def validate_gender(cls, v):
        allowed_genders = ['Male', 'Female']
        if v not in allowed_genders:
            raise ValueError(f'Gender debe ser uno de: {allowed_genders}')
        return v
    
    @validator('partner', 'dependents', 'phone_service')
    def validate_yes_no_fields(cls, v, field):
        allowed_values = ['Yes', 'No']
        if v not in allowed_values:
            raise ValueError(f'{field.name} debe ser "Yes" o "No"')
        return v

    class Config:
        # Configuración del modelo Pydantic
        str_strip_whitespace = True  # Eliminar espacios en blanco
        validate_assignment = True   # Validar en asignaciones
        use_enum_values = True      # Usar valores de enum
        schema_extra = {
            "example": {
                "gender": "Female",
                "senior_citizen": 0,
                "partner": "Yes",
                "dependents": "No", 
                "tenure": 24,
                "phone_service": "Yes",
                "internet_service": "Fiber optic",
                "online_security": "No",
                "tech_support": "Yes",
                "contract": "Month-to-month",
                "payment_method": "Electronic check",
                "monthly_charges": 85.50,
                "total_charges": 2052.00,
                "customer_id": "CUST001"
            }
        }

class PredictionResponse(BaseModel):
    """Respuesta de la predicción de churn"""
    
    prediction: int = Field(
        ...,
        description="Predicción de churn (0=No Churn, 1=Churn)",
        example=1
    )
    
    churn_probability: float = Field(
        ...,
        ge=0.0,
        le=1.0,
        description="Probabilidad de churn",
        example=0.75
    )
    
    retention_probability: float = Field(
        ...,
        ge=0.0,
        le=1.0,
        description="Probabilidad de retención",
        example=0.25
    )
    
    risk_category: RiskCategory = Field(
        ...,
        description="Categoría de riesgo",
        example="High"
    )
    
    confidence: float = Field(
        ...,
        ge=0.0,
        le=1.0,
        description="Confianza del modelo",
        example=0.75
    )
    
    model_info: Dict[str, Any] = Field(
        ...,
        description="Información del modelo utilizado",
        example={
            "name": "churn_logistic_regression",
            "type": "Pipeline",
            "version": "1.0.0"
        }
    )
    
    processing_time_ms: int = Field(
        ...,
        ge=0,
        description="Tiempo de procesamiento en milisegundos",
        example=150
    )
    
    timestamp: datetime = Field(
        default_factory=datetime.now,
        description="Timestamp de la predicción",
        example="2024-12-01T10:30:00"
    )

class BatchPredictionRequest(BaseModel):
    """Solicitud de predicción por lotes"""
    
    customers: List[CustomerInput] = Field(
        ...,
        min_items=1,
        max_items=1000,  # Límite para evitar sobrecarga
        description="Lista de clientes para predecir"
    )
    
    include_details: bool = Field(
        True,
        description="Incluir detalles completos en la respuesta"
    )
```


## 6. Contenedorización con Docker (...)

### Dockerfile Optimizado

```dockerfile
# Dockerfile multi-stage optimizado para FastAPI + ML con uv

# Etapa 1: Builder - Instalar dependencias
FROM python:3.12-slim as builder

# Instalar uv (gestor de paquetes rápido)
COPY --from=ghcr.io/astral-sh/uv:latest /uv /usr/local/bin/uv

# Variables de entorno para optimización
ENV UV_COMPILE_BYTECODE=1
ENV UV_LINK_MODE=copy
ENV PYTHONPATH=/app
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# Directorio de trabajo
WORKDIR /app

# Copiar archivos de dependencias
COPY pyproject.toml ./

# Crear entorno virtual e instalar dependencias
RUN uv venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"
RUN uv pip install -r pyproject.toml

# Etapa 2: Runtime - Aplicación final
FROM python:3.12-slim as runtime

# Instalar dependencias del sistema necesarias para ML
RUN apt-get update && apt-get install -y \
    libgomp1 \
    && rm -rf /var/lib/apt/lists/*

# Variables de entorno
ENV PYTHONPATH=/app
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1
ENV PATH="/opt/venv/bin:$PATH"

# Crear usuario no-root para seguridad
RUN groupadd -r appuser && useradd -r -g appuser appuser

# Directorio de trabajo
WORKDIR /app

# Copiar entorno virtual desde builder
COPY --from=builder /opt/venv /opt/venv

# Copiar código de la aplicación
COPY src/ /app/src/
COPY models/ /app/models/

# Crear directorio para logs
RUN mkdir -p /app/logs && chown -R appuser:appuser /app

# Cambiar a usuario no-root
USER appuser

# Health check
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \
    CMD python -c "import requests; requests.get('http://localhost:8000/health')"

# Puerto de la aplicación
EXPOSE 8000

# Comando por defecto - usar FastAPI CLI
CMD ["fastapi", "run", "src/main.py", "--host", "0.0.0.0", "--port", "8000"]
```

### Docker Compose para Desarrollo

```yaml
# docker-compose.yml
version: '3.8'

services:
  # Servicio principal de la API
  churn-api:
    build:
      context: .
      dockerfile: Dockerfile
    container_name: churn-prediction-api
    ports:
      - "8000:8000"
    environment:
      - ENVIRONMENT=development
      - LOG_LEVEL=INFO
      - MODELS_PATH=/app/models
    volumes:
      # Volumen para desarrollo - hot reload
      - ./src:/app/src:ro
      - ./models:/app/models:ro
      - ./logs:/app/logs
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 30s
    networks:
      - ml-network

  # Redis para caché (opcional)
  redis:
    image: redis:7-alpine
    container_name: churn-api-cache
    ports:
      - "6379:6379"
    volumes:
      - redis_data:/data
    networks:
      - ml-network
    restart: unless-stopped
    command: redis-server --appendonly yes

volumes:
  redis_data:

networks:
  ml-network:
    driver: bridge
```


## 7. Despliegue en la Nube con Fly.io (...)

### Configuración de Fly.io

```toml
# fly.toml
app = "churn-prediction-api"
primary_region = "mad"  # Madrid - cambiar según tu preferencia

# Build configuration
[build]

# Environment variables
[env]
  PORT = "8000"
  ENVIRONMENT = "production"
  LOG_LEVEL = "INFO"

# HTTP service configuration
[http_service]
  internal_port = 8000
  force_https = true
  auto_stop_machines = true
  auto_start_machines = true
  min_machines_running = 0
  processes = ["app"]

  # Health checks
  [[http_service.checks]]
    grace_period = "10s"
    interval = "30s"
    method = "GET"
    timeout = "5s"
    path = "/health"
    protocol = "http"

# Machine configuration
[[vm]]
  memory = "2gb"      # Suficiente para modelos ML
  cpu_kind = "shared"
  cpus = 1
  processes = ["app"]

# Configuración de procesos
[processes]
  app = "fastapi run src/main.py --host 0.0.0.0 --port 8000"
```

### Script de Deployment

```bash
#!/bin/bash
# scripts/deploy-fly.sh

# Script de deployment para Fly.io

set -e

echo "🚀 Iniciando deployment a Fly.io"

# Variables
FLY_APP_NAME="churn-prediction-api"
REGION="mad"  # Madrid

# Verificar que flyctl esté instalado
check_flyctl() {
    if ! command -v flyctl &> /dev/null; then
        echo "❌ flyctl no está instalado"
        echo "Instala flyctl desde: https://fly.io/docs/hands-on/install-flyctl/"
        exit 1
    fi
    echo "✅ flyctl instalado"
}

# Verificar autenticación
check_auth() {
    if ! flyctl auth whoami &> /dev/null; then
        echo "⚠️ No estás autenticado en Fly.io"
        echo "Ejecutando 'flyctl auth login'..."
        flyctl auth login
    fi
    echo "✅ Autenticado en Fly.io"
}

# Verificar que los modelos existen
check_models() {
    if [ ! -d "models" ] || [ -z "$(ls -A models/*.joblib 2>/dev/null)" ]; then
        echo "❌ No se encontraron modelos entrenados"
        exit 1
    fi
    echo "✅ Modelos encontrados"
}

# Crear aplicación si no existe
create_or_update_app() {
    if flyctl apps show $FLY_APP_NAME &> /dev/null; then
        echo "✅ Aplicación '$FLY_APP_NAME' ya existe"
    else
        echo "📱 Creando nueva aplicación..."
        flyctl apps create $FLY_APP_NAME --region $REGION
        echo "✅ Aplicación creada"
    fi
}

# Deploy de la aplicación
deploy_app() {
    echo "🚀 Iniciando deployment..."
    flyctl deploy --remote-only --strategy immediate
    echo "✅ Deployment completado"
}

# Verificar que el deployment funcionó
verify_deployment() {
    local app_url="https://${FLY_APP_NAME}.fly.dev"
    
    echo "🔍 Verificando deployment..."
    sleep 10
    
    if curl -f "${app_url}/health" > /dev/null 2>&1; then
        echo "✅ ¡Deployment exitoso!"
        echo "📖 Documentación API: ${app_url}/docs"
        echo "🔍 Health check: ${app_url}/health"
        return 0
    else
        echo "❌ Deployment falló"
        flyctl logs --app $FLY_APP_NAME
        return 1
    fi
}

# Función principal
main() {
    echo "════════════════════════════════════════"
    echo "   🚀 DEPLOYMENT A FLY.IO"
    echo "════════════════════════════════════════"
    
    check_flyctl
    check_auth
    check_models
    create_or_update_app
    deploy_app
    
    if verify_deployment; then
        echo "🎉 ¡Deployment completado exitosamente!"
    else
        echo "💥 Deployment falló. Revisa los logs."
        exit 1
    fi
}

# Ejecutar función principal
main "$@"
```

## 8. Testing de la API (...)

### Tests Automatizados con Pytest

```python
# tests/test_api.py
import pytest
import httpx
from fastapi.testclient import TestClient
import json
import os
import sys

# Configurar path para imports
sys.path.insert(0, os.path.join(os.path.dirname(__file__), '..', 'src'))

from main import app

# Cliente de testing
client = TestClient(app)

class TestHealthEndpoints:
    """Tests para endpoints de salud y monitoreo."""
    
    def test_root_endpoint(self):
        """Test del endpoint raíz."""
        response = client.get("/")
        assert response.status_code == 200
        
        data = response.json()
        assert data["message"] == "🎯 Churn Prediction API"
        assert data["version"] == "1.0.0"
        assert "status" in data
    
    def test_health_check(self):
        """Test del health check básico."""
        response = client.get("/health")
        assert response.status_code == 200
        
        data = response.json()
        assert data["status"] == "healthy"
        assert "models_available" in data

class TestPredictionEndpoints:
    """Tests para endpoints de predicción."""
    
    @pytest.fixture
    def valid_customer_data(self):
        """Datos válidos de cliente para testing."""
        return {
            "gender": "Female",
            "senior_citizen": 0,
            "partner": "Yes",
            "dependents": "No",
            "tenure": 24,
            "phone_service": "Yes",
            "internet_service": "Fiber optic",
            "online_security": "No",
            "tech_support": "Yes",
            "contract": "Month-to-month",
            "payment_method": "Electronic check",
            "monthly_charges": 85.50,
            "total_charges": 2052.00,
            "customer_id": "TEST001"
        }
    
    def test_predict_valid_customer(self, valid_customer_data):
        """Test de predicción con datos válidos."""
        response = client.post("/predict", json=valid_customer_data)
        assert response.status_code == 200
        
        data = response.json()
        
        # Verificar estructura de la respuesta
        required_fields = [
            "prediction", "churn_probability", "retention_probability",
            "risk_category", "confidence", "model_info", 
            "processing_time_ms", "timestamp"
        ]
        
        for field in required_fields:
            assert field in data, f"Campo {field} faltante en respuesta"
        
        # Verificar tipos y rangos
        assert isinstance(data["prediction"], int)
        assert data["prediction"] in [0, 1]
        
        assert 0 <= data["churn_probability"] <= 1
        assert 0 <= data["retention_probability"] <= 1
        
        assert data["risk_category"] in ["Low", "Medium", "High"]
        assert data["processing_time_ms"] > 0
    
    def test_predict_invalid_data(self):
        """Test con datos inválidos."""
        invalid_data = {
            "gender": "Other",  # No permitido
            "senior_citizen": 2,  # Fuera de rango
            "tenure": -5,  # Negativo
            "monthly_charges": 0  # Debe ser > 0
        }
        
        response = client.post("/predict", json=invalid_data)
        assert response.status_code == 422  # Unprocessable Entity

# Script para ejecutar tests
if __name__ == "__main__":
    pytest.main([__file__])
```

## 9. Mejores Prácticas y Tips (...)

### Checklist de Producción

```
# 📋 CHECKLIST DE PRODUCCIÓN PARA API DE ML

## ✅ Código y Arquitectura
- [ ] Código limpio y bien documentado
- [ ] Separación clara entre entrenamiento e inferencia
- [ ] Pipelines de scikit-learn para consistencia
- [ ] Validación robusta con Pydantic
- [ ] Manejo de errores comprehensivo
- [ ] Logging estructurado configurado
- [ ] Tests unitarios e integración (>80% cobertura)

## ✅ Modelo y Datos
- [ ] Modelo validado en datos de prueba
- [ ] Métricas de rendimiento documentadas
- [ ] Versionado de modelos implementado
- [ ] Backup de modelos configurado

## ✅ API y Rendimiento
- [ ] Documentación API completa (Swagger)
- [ ] Health checks funcionando
- [ ] CORS configurado apropiadamente
- [ ] Timeouts configurados

## ✅ Seguridad
- [ ] Variables sensibles en variables de entorno
- [ ] Usuario no-root en contenedor Docker
- [ ] HTTPS habilitado
- [ ] Validación de entrada estricta

## ✅ Infraestructura
- [ ] Dockerfile optimizado (multi-stage)
- [ ] Imagen Docker pequeña y eficiente
- [ ] Monitoreo y alertas configurados
- [ ] Backup y recuperación probados
```

### Optimizaciones de Rendimiento

```python
# Técnicas de optimización para APIs de ML

# 1. Cache de modelos
from functools import lru_cache

@lru_cache(maxsize=None)
def load_model_cached(model_path: str):
    """Cargar modelo con cache para evitar recargas."""
    return joblib.load(model_path)

# 2. Procesamiento por lotes
@app.post("/predict/batch")
async def batch_predict(requests: List[PredictionRequest]):
    """Procesar múltiples predicciones eficientemente."""
    # Preparar datos para predicción vectorizada
    batch_data = [req.features.dict() for req in requests]
    
    # Predicción vectorizada (más eficiente)
    predictions = model.predict(batch_data)
    probabilities = model.predict_proba(batch_data)
    
    return results

# 3. Async/Await para I/O
async def log_prediction_async(prediction_data: dict):
    """Logging asíncrono para no bloquear requests."""
    async with aiofiles.open("predictions.log", "a") as f:
        await f.write(f"{json.dumps(prediction_data)}\n")

# 4. Configuración de Uvicorn para producción
if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=8000,
        workers=4,  # Número de workers basado en CPU
        loop="uvloop",  # Loop más rápido
        http="httptools",  # Parser HTTP más rápido
    )
```


## 10. Conclusiones y Próximos Pasos

### Resumen de lo Implementado

🎓 **Has aprendido a implementar un sistema completo de ML en producción:**

1. **Gestión moderna de proyectos** con UV y pyproject.toml
2. **Machine Learning pipelines** con scikit-learn
3. **API robusta** con FastAPI y validación Pydantic
4. **Contenedorización** optimizada con Docker
5. **Deployment** en la nube con Fly.io
6. **Testing automatizado** con pytest
7. **Monitoreo** y observabilidad

### Valor Agregado vs. Enfoques Tradicionales

| Aspecto | Tradicional (Flask + pip) | Moderno (FastAPI + uv) |
|---------|---------------------------|------------------------|
| **Velocidad API** | ~1000 req/s | ~3000+ req/s |
| **Documentación** | Manual | Automática |
| **Validación** | Manual | Automática |
| **Install deps** | pip install (30s) | uv sync (3s) |
| **Typing** | Opcional | Nativo |
| **Async** | Complejo | Nativo |

### Próximos Pasos Recomendados

🛣️ **Extensiones avanzadas:**

1. **MLOps**: Integrar MLflow para model registry
2. **Monitoreo**: Añadir Prometheus + Grafana
3. **Seguridad**: Implementar autenticación JWT
4. **Escalabilidad**: Añadir Redis cache y load balancing
5. **CI/CD**: GitHub Actions para deployment automático

### Comandos Finales

```bash
# Configuración inicial
uv sync                                 # Instalar dependencias
python scripts/setup.py               # Configurar proyecto

# Desarrollo local
uv run uvicorn src.main:app --reload  # Servidor desarrollo

# Testing
pytest tests/ --cov=src               # Tests con cobertura

# Deployment
./scripts/deploy-fly.sh              # Deploy a producción
```

### Estructura Final del Proyecto

```
mi-proyecto-ml/
├── src/                    # Código fuente
│   ├── main.py            # Aplicación FastAPI
│   ├── schemas/           # Modelos Pydantic
│   └── ml/                # Lógica ML
├── models/                # Modelos entrenados (.joblib)
├── tests/                 # Tests automatizados
├── scripts/               # Scripts de utilidad
├── Dockerfile             # Contenedorización
├── fly.toml              # Config Fly.io
└── pyproject.toml        # Dependencias UV
```

🎉 **¡Felicitaciones! Has implementado una API de ML moderna, escalable y lista para producción usando las mejores prácticas y herramientas de 2025.**

Tu API ahora puede:
- ✅ Servir predicciones de ML a miles de usuarios
- ✅ Validar datos automáticamente
- ✅ Documentarse a sí misma
- ✅ Desplegarse globalmente en segundos
- ✅ Monitorearse y alertar automáticamente
- ✅ Escalarse según demanda

**¡Es hora de llevarlo a producción y ver tu modelo en acción!** 🚀

> **Nota**  
> Esta notebook se inspiró en el workshop [**mlzoomcamp-fastapi-uv**](https://github.com/alexeygrigorev/workshops/tree/main/mlzoomcamp-fastapi-uv), ofrecido por *Alexey Grigorev*, fundador de **DataTalks.Club**. 
> Agradecimientos a la comunidad por compartir estos recursos abiertos.
> Además, me tomé la molestia de guardar los archivos del workshop en la carpeta **`workshop_fastapi_ml`** para que tengas acceso rápido al material de referencia.